# Hands - On 2 - Programação Dinâmica com iteração por Policy ou Valor

Este hands-on é destinado a aplicação da solução via programação dinâmica das equações de Bellman para um processo de decisão de Markov. O ambiente que será usado para a solução é o *grid world* como apresentado abaixo

![alt text](rlimages/grid01.png)


nosso agente será um robô que será inserido neste ambiente e que o processo de aprendizagem (episódio) acaba que o robô alcança o estado (4,3), com recompensa +1, e o estado (4,2), com recompensa -1. Os valores de -0.02 nos demais estados são custos que representam o custo associado ao movimento do robô (consumo de bateria). O objetivo é fazer o robô chegar ao estado (4,3) com o menor custo possível (maior recompensa).  
***
Relembrando, a solução via programação dinâmica pode ocorrer de dois modos: **Iteração por valor** ou **Iteração por Policy**.

Na **iteração por valor** segue o seguinte algoritmo:
*******
1. Inicialize V(s) = 0 para todos os estados s
2. Para todo s, atualize:
    ![alt text](rlimages/value_ite.png)  
3. Repetidamente computando o passo 2, haverá convergência para o valor ótimo de cada estado:
    ![alt text](rlimages/value_conv.png)
    
ao fim do processo de iteração o agente terá aprendido a dinâmica do ambiente e poderá realizar sempre a melhor ação dado o estado que ele se encontra. Para isso, basta a partir do valor final de V(s) montar a policy ótima que é dado como:
    ![alt text](rlimages/policy_value.png)
*****

Já na **iteração por policy** segue o seguinte algoritmo:
*******
1. Inicialize uma policy aleatória ∏
2. Repetidamente faça o seguinte até a convergência:
    Resolva a equação de Bellman para o policy atual
    ![alt text](rlimages/policy.png)  
3. Calcule a policy como se fosse a ótima para a função valor V(s) calculada:
    ![alt text](rlimages/policy_value.png)
4. Repita os passos e ambos, o valor e policy, irão convergir para o valor ótimo
    ![alt text](rlimages/policy_conv.png)
    
*****

Antes de entrarmos de fato na código da resolução, vamos relembrar que usaremos a seguinte dinâmica para as probabilidades de transição:
    ![alt text](rlimages/dinamica.png)

essa dinâmica indica que cada vez que o agente escolhe uma certa ação, por exemplo ir para o norte, Há 80% de chances dele ir de fato para o norte, porém há 10% de chance dele ir para o leste (direita) ou 10% dele ir para o oeste (esquerda).

Exposto isso, vamos à aplicação da solução via programação dinâmica. Primeiramente vamos definir algumas funções auxiliares para a movimentação do agente:

In [1]:
import random, operator

def argmax(seq, fn): 

    best = seq[0]; best_score = fn(best)

    for x in seq:
        x_score = fn(x)
        if x_score > best_score:
            best, best_score = x, x_score
    return best

def vector_add(a, b): #adicao sem precisar da numpy

    return tuple(map(operator.add, a, b))


#orientacoes: leste, norte, oeste, sul
orientations = [(1, 0), (0, 1), (-1, 0), (0, -1)]

def turn_right(orientation):

    return orientations[orientations.index(orientation)-1]

def turn_left(orientation):
    
    return orientations[ (orientations.index(orientation)+1) % len(orientations)]	

def isnumber(x): 
    return hasattr(x, '__int__')


com as funções auxiliares criadas, vamos agora criar uma classe que representará a MDP (Markov Decision process - Processo de Decisão Markov) 

In [2]:
class MDP:

    def __init__(self, init_pos, actlist, terminals, transitions={}, states=None, gamma=0.99):

        if not (0 < gamma <= 1):
            raise ValueError('MDP deve ter 0 < gamma <=1')
        if states:
            self.states = states # estados
        else:
            self.states = set() 
        
        self.init_pos = init_pos # posicao inicial
        self.actlist = actlist # lista de acoes
        self.terminals = terminals # Estado final - objetivo
        self.transitions = transitions # transicoes de estados
        self.gamma = gamma # gamma
        self.reward = {} # recompensa 

    def R(self, state): #retorna a recompensa dado o estado

        return self.reward[state]

    def T(self, state, action): #retorna a a prob. de transicao dado a acao e o estado atual

        if(self.transitions == {}):

            raise ValueError("Modelo de transicao nao definido")
        else:
            return self.transitions[state][action]

    def actions(self, state): #retorna a acao dado o estado

        if state in self.terminals:
            return [None]
        else:
            return self.actlist

Agora definimos um classe para representar o grid que é o ambiente estará inserido. Essa classe do grid será derivada da classe MDP, pois como já se sabe o ambiente e seus estados são elementos da processo de decisão

In [3]:
class GridMDP(MDP):

    def __init__(self, grid, terminals, init_pos=(0,0), gamma = 0.99):

        grid.reverse() # reverte os indices para ficar de acordo com a figura do grid apresentado no começo

        #inicializacao do grid
        MDP.__init__(self, init_pos, actlist=orientations, terminals=terminals, gamma=gamma)
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0])
        for x in range(self.cols):
            for y in range(self.rows):
                self.reward[x,y] = grid[y][x]
                if grid[y][x] is not None:
                    self.states.add( (x, y) )

    def T(self, state, action):

        if action is None:
            return [(0.0, state)]
        else:

            return [(0.8, self.go(state,action)), 
                    (0.1, self.go(state, turn_right(action))), 
                    (0.1, self.go(state, turn_left(action)))]

    def go(self, state, direction): #retorna o proximo estado dado o estado atual e direcao indicada pela acaoo

        state1 = vector_add(state, direction)
        return state1 if state1 in self.states else state

    def to_grid(self, mapping): #auxiliar para o grid sem peso para o problema em si

        return list(reversed([[mapping.get((x,y), None) for x in range(self.cols)] for y in range(self.rows)]))

    def to_arrows(self, policy): # retorna o grid com arrows indicando a policy ótima
        chars = { (1,0): '>', (0,1): '^', (-1,0): '<', (0,-1): 'v', None:'.'}
        return self.to_grid({s: chars[a] for (s, a) in policy.items()})

Por fim, vamos agora definir as funções para resolução por valor e por policy bem como algumas outras funções auxiliares

In [4]:
def value_iteration(mdp, epsilon=0.001): #iteração por valor - primeira opcao de resolucao

    STSN = {s: 0 for s in mdp.states}

    R, T, gamma = mdp.R, mdp.T, mdp.gamma

    while True:
            
        STS = STSN.copy()
        delta = 0

        for s in mdp.states:

            STSN[s] = R(s) + gamma * max([sum([p * STS[s1] for (p, s1) in T(s, a)]) for a in mdp.actions(s)])
            delta = max(delta, abs(STSN[s] - STS[s]))

            if delta < 	epsilon * (1 - gamma)/ gamma:
                return STS

def best_policy(mdp, STS): #retorna a melhor policy dado a MDP e a função valor

    pi = {}

    for s in mdp.states:

        pi[s] = argmax(mdp.actions(s), lambda a: expected_utility(a, s, STS, mdp))

    return pi

def expected_utility(a, s, STS, mdp): #funcao para calculo da policy otima dado a funcao valor já definida

    return sum([p * STS[s1] for (p, s1) in mdp.T(s, a)])

def policy_iteration(mdp): #resolucao por policy

    STS = {s: 0 for s in mdp.states}
    pi = {s: random.choice(mdp.actions(s)) for s in mdp.states}

    while True:

        STS = policy_evaluation(pi, STS, mdp)
        unchanged = True
        for s in mdp.states:

            a = argmax(mdp.actions(s), lambda a: expected_utility(a, s, STS, mdp))

            if a != pi[s]:
                pi[s] = a
                unchanged = False

        if unchanged:
            return pi 

def policy_evaluation(pi, STS, mdp, k = 20): #dado a policy resolve o sistema via iteração por valor (resolucao por Belmman)

    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    for i in range(k):
        for s in mdp.states:
            STS[s] = R(s) + gamma * sum([p * STS[s1] for (p, s1) in T(s, pi[s])])

    return STS

def print_table(table, header = None, sep = '   ', numftm = '{}'): #auxiliar para imprimir o grid no console - Não importa para a resolução em si


    justs = ['rjust' if isnumber(x) else 'ljust' for x in table[0]] 
    if header:

        table.insert(0, header)

    table = [[numftm.format(x) if isnumber(x) else x for x in row] for row in table]

    sizes = list(map(lambda seq: max(map(len, seq)), list(zip(*[map(str, row) for row in table ]))))

    for row in table:
        print(sep.join(getattr(str(x), j)(size) for (j, size, x) in zip(justs, sizes, row)))


In [5]:
env_grid = [[-0.02, -0.02, -0.02, 1],
                [-0.02, None, -0.02, -1],
                [-0.02, -0.02, -0.02, -0.02]] # definicao do grid do ambiente

terminals = [(3,2), (3, 1)] # estados onde pode ocorrer o encerramento do episodio

sequential_dec_env = GridMDP(env_grid, terminals=terminals) 

epsilon = 0.01 #definir um limiar para convergencia

vl = value_iteration(sequential_dec_env, epsilon) # resolucao por valor
value_iter = best_policy(sequential_dec_env, vl) # best policy dado o valor ótimo
print('Optimal policy  based on  Value iteration \n')
print_table(sequential_dec_env.to_arrows(value_iter))

print('\n')
policy_iter = policy_iteration(sequential_dec_env) #resolucao por policy
print('Optimal policy  based on  policy iteration \n')
print_table(sequential_dec_env.to_arrows(policy_iter))

Optimal policy  based on  Value iteration 

>   >      >   .
^   None   ^   .
^   <      <   <


Optimal policy  based on  policy iteration 

>   >      >   .
^   None   ^   .
^   <      <   <


***
Agora, tire um tempo para enter bem como este código funciona. Foque nas funções da iteração por valor e iteração por policy. Depois de entendido a base de funcionamento faça o seguinte:

   1. Rode problema para alguns valores diferentes de gamma. Teste gamma com valores próximos de 0 e com valores próximos de 1
         - O que acontece com a policy ótima ao final de cada modificação? Por que acontece isso com a policy? 
   2. Rode o problema para alguns valores diferentes de epsilon. 
       - O que acontece com a policy ótima ao final de cada modificação? Por que acontece isso com a policy?
   3. Bônus - Desafio:
       - Quantas iterações são necessárias para a convergência na iteração por valor? Crie um lógica dentro da função value_iteration() para fazer essa contagem. Teste para vários valores de epsilon e gamma
         